# Анализ качества вина

## Описание данных

Датасет содержит результаты физико-химического анализа португальского вина сорта "Vinho Verde" (Зелёное вино) и экспертные оценки качества. Включает два отдельных файла для красного и белого вина.

**Название**: Wine Quality Dataset  
**Источник**: UCI Machine Learning Repository  
**URL**: https://archive.ics.uci.edu/dataset/186/wine+quality  
**Прямое скачивание**: https://archive.ics.uci.edu/static/public/186/wine+quality.zip  
**Лицензия**: CC BY 4.0  
**Авторы**: Paulo Cortez, António Cerdeira, Fernando Almeida, Telmo Matos, José Reis  
**Год публикации**: 2009  
**Статья**: "Modeling wine preferences by data mining from physicochemical properties"

## Состав датасета

### Файлы

| Файл | Размер | Описание |
|------|--------|----------|
| **winequality-red.csv** | 1,599 записей | Красное вино "Vinho Verde" |
| **winequality-white.csv** | 4,898 записей | Белое вино "Vinho Verde" |
| **Всего** | 6,497 записей | Объединенный датасет |

**Формат**: CSV с разделителем "точка с запятой" (;)

## Структура данных

### Входные переменные (физико-химические параметры)

| № | Столбец | Единицы измерения | Описание |
|---|---------|-------------------|----------|
| 1 | **fixed acidity** | г/дм³ | Фиксированная кислотность (винная кислота) |
| 2 | **volatile acidity** | г/дм³ | Летучая кислотность (уксусная кислота) |
| 3 | **citric acid** | г/дм³ | Лимонная кислота |
| 4 | **residual sugar** | г/дм³ | Остаточный сахар после ферментации |
| 5 | **chlorides** | г/дм³ | Содержание хлоридов (соли) |
| 6 | **free sulfur dioxide** | мг/дм³ | Свободный диоксид серы (SO₂) |
| 7 | **total sulfur dioxide** | мг/дм³ | Общий диоксид серы |
| 8 | **density** | г/см³ | Плотность вина |
| 9 | **pH** | — | Уровень кислотности (0-14 шкала) |
| 10 | **sulphates** | г/дм³ | Сульфаты (добавка, консервант) |
| 11 | **alcohol** | % vol | Содержание алкоголя |

### Выходная переменная

| Столбец | Тип | Диапазон | Описание |
|---------|-----|----------|----------|
| **quality** | int | 0-10 | Экспертная оценка качества (медиана минимум 3 оценок) |

## ИМПОРТ БИБЛИОТЕК

In [ ]:
import pandas as pd
import numpy as np

## ЗАГРУЗКА И ПЕРВИЧНЫЙ АНАЛИЗ

In [ ]:
# 1. Загрузка датасета красного вина
url_red = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
df_red = pd.read_csv(url_red, sep=';')

In [ ]:
# 2. Первые 10 строк
df_red.head(10)

In [ ]:
# 3. Информация о структуре данных
df_red.info()

In [ ]:
# 4. Описательная статистика
df_red.describe()

In [ ]:
# 5. Список всех колонок
df_red.columns.tolist()

In [ ]:
# 6. Размер датасета
print(f"Строк: {df_red.shape[0]}, Столбцов: {df_red.shape[1]}")

## ОБРАБОТКА ПРОПУСКОВ И ДУБЛИКАТОВ

In [ ]:
# 7. Количество пропущенных значений
df_red.isnull().sum()

In [ ]:
# 8. Процент пропущенных значений
(df_red.isnull().sum() / len(df_red) * 100).round(2)

In [ ]:
# 9. Проверка полных дубликатов
full_duplicates = df_red.duplicated().sum()
print(f"Количество полных дубликатов: {full_duplicates}")

In [ ]:
# 10. Дубликаты по всем столбцам кроме quality
cols_without_quality = [col for col in df_red.columns if col != 'quality']
partial_duplicates = df_red.duplicated(subset=cols_without_quality).sum()
print(f"Дубликаты без учета quality: {partial_duplicates}")

In [ ]:
# 11. Удаление дубликатов
df_red = df_red.drop_duplicates()
print(f"\nРазмер после удаления дубликатов: {df_red.shape}")

In [ ]:
# 12. Распределение оценок quality
df_red['quality'].value_counts().sort_index()

## ПРЕОБРАЗОВАНИЕ ТИПОВ ДАННЫХ

In [ ]:
# 13. Убедимся, что химические параметры имеют тип float
chemical_cols = [col for col in df_red.columns if col != 'quality']
for col in chemical_cols:
    df_red[col] = df_red[col].astype(float)

In [ ]:
# 14. quality в int
df_red['quality'] = df_red['quality'].astype(int)

In [ ]:
# 15. Создание копии датасета
df = df_red.copy()

In [ ]:
# Проверка типов данных
df.dtypes

## СОЗДАНИЕ НОВЫХ ПРИЗНАКОВ

In [ ]:
# 16. quality_category
def categorize_quality(quality):
    if quality <= 4:
        return 'Poor'
    elif quality <= 6:
        return 'Average'
    elif quality <= 8:
        return 'Good'
    else:
        return 'Excellent'

df['quality_category'] = df['quality'].apply(categorize_quality)

In [ ]:
# 17. alcohol_level
def categorize_alcohol(alcohol):
    if alcohol < 10:
        return 'Low'
    elif alcohol <= 12:
        return 'Medium'
    else:
        return 'High'

df['alcohol_level'] = df['alcohol'].apply(categorize_alcohol)

In [ ]:
# 18. total_acidity
df['total_acidity'] = df['fixed acidity'] + df['volatile acidity'] + df['citric acid']

In [ ]:
# 19. sulfur_ratio
df['sulfur_ratio'] = df['free sulfur dioxide'] / df['total sulfur dioxide']

In [ ]:
# 20. sweetness_category
def categorize_sweetness(sugar):
    if sugar < 4:
        return 'Dry'
    elif sugar <= 12:
        return 'Off-Dry'
    else:
        return 'Sweet'

df['sweetness_category'] = df['residual sugar'].apply(categorize_sweetness)

In [ ]:
# 21. acidity_level
def categorize_ph(ph):
    if ph < 3.0:
        return 'Low_pH'
    elif ph <= 3.4:
        return 'Medium_pH'
    else:
        return 'High_pH'

df['acidity_level'] = df['pH'].apply(categorize_ph)

In [ ]:
# 22. chlorides_category (на основе квартилей)
q25 = df['chlorides'].quantile(0.25)
q75 = df['chlorides'].quantile(0.75)

def categorize_chlorides(value):
    if value <= q25:
        return 'Low'
    elif value <= q75:
        return 'Medium'
    else:
        return 'High'

df['chlorides_category'] = df['chlorides'].apply(categorize_chlorides)

In [ ]:
# 23. is_high_quality
df['is_high_quality'] = df['quality'] >= 7

In [ ]:
# 24. balance_score (нормализация и комбинация параметров)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
balance_features = ['alcohol', 'pH', 'total_acidity', 'residual sugar']
normalized = scaler.fit_transform(df[balance_features])
df['balance_score'] = normalized.mean(axis=1)

In [ ]:
# 25. Первые 10 строк с новыми столбцами
df.head(10)

## ФИЛЬТРАЦИЯ И СОРТИРОВКА ДАННЫХ

In [ ]:
# 26-27. Вина высокого качества
high_quality_wines = df[df['quality'] >= 7]
print(f"Количество: {len(high_quality_wines)}")

In [ ]:
# 28. Высокий алкоголь и низкая кислотность
high_alc_low_acid = df[(df['alcohol'] > 12) & (df['pH'] > 3.4)]
print("ВЫСОКИЙ АЛКОГОЛЬ И НИЗКАЯ КИСЛОТНОСТЬ")
print(f"Количество: {len(high_alc_low_acid)}")

In [ ]:
# 29. Query
premium_wines = df.query('alcohol > 12 and quality >= 7 and pH < 3.5')
print("ПРЕМИУМ ВИНА (алкоголь > 12, качество >= 7, pH < 3.5)")
print(f"Количество премиальных вин: {len(premium_wines)}")

In [ ]:
# 30. Сортировка по качеству
sorted_by_quality = df.sort_values('quality', ascending=False)
sorted_by_quality.head(10)

In [ ]:
# 31. Сортировка по alcohol, затем quality
sorted_multi = df.sort_values(['alcohol', 'quality'], ascending=[False, False])
sorted_multi.head(10)

In [ ]:
# 32. Топ-50 по содержанию алкоголя
top_50_alcohol = df.nlargest(50, 'alcohol')
top_50_alcohol[['alcohol', 'quality', 'pH']].head(10)

## ГРУППИРОВКА И АГРЕГАЦИЯ

In [ ]:
# 33. Количество вин по quality
wines_by_quality = df.groupby('quality').size()
wines_by_quality

In [ ]:
# 34. Средние значения химических параметров по качеству
chemical_cols_only = ['fixed acidity', 'volatile acidity', 'citric acid', 
                      'residual sugar', 'chlorides', 'free sulfur dioxide', 
                      'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
avg_by_quality = df.groupby('quality')[chemical_cols_only].mean()
avg_by_quality.round(2)

In [ ]:
# 35. Оценка с наибольшим количеством образцов
most_common_quality = wines_by_quality.idxmax()
print(f"Наиболее распространенная оценка качества: {most_common_quality}, Количество: {wines_by_quality[most_common_quality]}")

In [ ]:
# 36. Группировка по quality_category
category_stats = df.groupby('quality_category').agg({
    'quality': 'count',
    'alcohol': ['mean', 'std'],
    'pH': 'mean',
    'total_acidity': 'mean'
})
category_stats.columns = ['Количество', 'Средний алкоголь', 'СКО алкоголя', 
                          'Средний pH', 'Средняя кислотность']

category_stats.round(2)

In [ ]:
# 37. Агрегация по alcohol_level
alcohol_level_agg = df.groupby('alcohol_level').agg({
    'quality': ['mean', 'min', 'max', 'std'],
    'fixed acidity': ['mean', 'std'],
    'volatile acidity': ['mean', 'std'],
    'pH': 'mean',
    'sulphates': 'mean'
})

alcohol_level_agg.round(2)

In [ ]:
# 38. Корреляция с качеством
correlation_with_quality = df[chemical_cols_only].corrwith(df['quality']).sort_values(ascending=False)
correlation_with_quality.round(3)

## СВОДНЫЕ ТАБЛИЦЫ

In [ ]:
# 39. Сводная таблица: quality_category × alcohol_level
pivot_quality_alcohol = pd.pivot_table(
    df,
    values='quality',
    index='quality_category',
    columns='alcohol_level',
    aggfunc='count',
    fill_value=0
)

pivot_quality_alcohol.round(2)

In [ ]:
# 40. Средний alcohol: sweetness_category × acidity_level
pivot_sweet_acid = pd.pivot_table(
    df,
    values='alcohol',
    index='sweetness_category',
    columns='acidity_level',
    aggfunc='mean'
)

pivot_sweet_acid.round(2)

In [ ]:
# 41. Процент high_quality: alcohol_level × acidity_level
pivot_high_quality = pd.pivot_table(
    df,
    values='is_high_quality',
    index='alcohol_level',
    columns='acidity_level',
    aggfunc='mean'
) * 100

pivot_high_quality.round(2)

In [ ]:
# 42. С итоговыми значениями
pivot_with_margins = pd.pivot_table(
    df,
    values='quality',
    index='alcohol_level',
    columns='quality_category',
    aggfunc='mean',
    margins=True
)

pivot_with_margins.round(2)

In [ ]:
# 43. Crosstab: УРОВЕНЬ АЛКОГОЛЯ × КАТЕГОРИЯ КАЧЕСТВА
crosstab_result = pd.crosstab(df['alcohol_level'], df['quality_category'])
crosstab_result

## ОБЪЕДИНЕНИЕ ДАТАФРЕЙМОВ

In [ ]:
# 44. Загрузка белого вина
url_white = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'
df_white = pd.read_csv(url_white, sep=';')

In [ ]:
# 45. Добавление столбца wine_type
df['wine_type'] = 'red'
df_white['wine_type'] = 'white'

In [ ]:
# Применим те же преобразования к белому вину
df_white['quality'] = df_white['quality'].astype(int)
df_white['quality_category'] = df_white['quality'].apply(categorize_quality)
df_white['alcohol_level'] = df_white['alcohol'].apply(categorize_alcohol)
df_white['total_acidity'] = df_white['fixed acidity'] + df_white['volatile acidity'] + df_white['citric acid']
df_white['sulfur_ratio'] = df_white['free sulfur dioxide'] / df_white['total sulfur dioxide']
df_white['sweetness_category'] = df_white['residual sugar'].apply(categorize_sweetness)
df_white['acidity_level'] = df_white['pH'].apply(categorize_ph)
df_white['is_high_quality'] = df_white['quality'] >= 7

In [ ]:
# 46. Объединение красных и белых вин
df_combined = pd.concat([df, df_white], ignore_index=True)
print(f"Всего вин: {len(df_combined)}")
print(f"Красных: {len(df_combined[df_combined['wine_type'] == 'red'])}")
print(f"Белых: {len(df_combined[df_combined['wine_type'] == 'white'])}")

In [ ]:
# 47. Эталонные значения для каждой категории
quality_benchmarks = df_combined.groupby('quality_category')[chemical_cols_only].mean().reset_index()
quality_benchmarks.columns = ['quality_category'] + [f'benchmark_{col}' for col in chemical_cols_only]
quality_benchmarks.round(2)

In [ ]:
# 48. Статистика по химическим параметрам
chemical_stats = pd.DataFrame({
    'parameter': chemical_cols_only,
    'mean': df_combined[chemical_cols_only].mean(),
    'std': df_combined[chemical_cols_only].std(),
    'min': df_combined[chemical_cols_only].min(),
    'max': df_combined[chemical_cols_only].max()
})

chemical_stats.round(2)

In [ ]:
# 49. Объединение с эталонными значениями
df_with_benchmarks = df_combined.merge(quality_benchmarks, on='quality_category', how='left')
df_with_benchmarks.head(10)

In [ ]:
# 50. Разделение обратно по типу вина
df_red_enriched = df_with_benchmarks[df_with_benchmarks['wine_type'] == 'red']
df_white_enriched = df_with_benchmarks[df_with_benchmarks['wine_type'] == 'white']

In [ ]:
# 51. Сравнение красных и белых вин
comparison = df_combined.groupby('wine_type')[chemical_cols_only + ['quality']].mean()
comparison.round(2)

## КОРРЕЛЯЦИОННЫЙ АНАЛИЗ

In [ ]:
# 52. Корреляционная матрица
correlation_matrix = df[chemical_cols_only + ['quality']].corr()
correlation_matrix.round(3)

In [ ]:
# 53. Наибольшая положительная корреляция с quality
positive_corr = correlation_matrix['quality'].drop('quality').sort_values(ascending=False)
positive_corr.head(5).round(3)

In [ ]:
# 54. Наибольшая отрицательная корреляция с quality
negative_corr = correlation_matrix['quality'].drop('quality').sort_values()
negative_corr.head(5).round(3)

## СОХРАНЕНИЕ РЕЗУЛЬТАТОВ

In [ ]:
# 55. Сохранение в Excel
with pd.ExcelWriter('wine_quality_analysis.xlsx', engine='openpyxl') as writer:
    # Лист 1: Красные вина с новыми признаками
    df.to_excel(writer, sheet_name='Red_Wine', index=False)
    
    # Лист 2: Вина высокого качества
    high_quality_wines.to_excel(writer, sheet_name='High_Quality', index=False)
    
    # Лист 3: Группировка по качеству
    avg_by_quality.to_excel(writer, sheet_name='By_Quality')
    
    # Лист 4: Группировка по алкоголю
    alcohol_level_agg.to_excel(writer, sheet_name='By_Alcohol')
    
    # Лист 5: Анализ химических параметров
    chemical_stats.to_excel(writer, sheet_name='Chemical_Analysis', index=False)
    
    # Лист 6: Корреляционная матрица
    correlation_matrix.to_excel(writer, sheet_name='Correlation_Matrix')
    
    # Лист 7: Профиль качественного вина
    quality_profile = df[df['is_high_quality']][chemical_cols_only].describe()
    quality_profile.to_excel(writer, sheet_name='Quality_Profile')
    
    # Лист 8: Сравнение красных и белых вин
    comparison.to_excel(writer, sheet_name='Red_vs_White')